# difflib package (method 1) - Haoran Wang

In [1]:
from src.difflib import find_matches
from src.difflib import match_all

from src.loader_difflib import data_preparation

In [2]:
left, right = data_preparation()

In [3]:
cities_right = right['city'].unique()
right_list = cities_right.tolist()

In [ ]:
output_difflib_1 = match_all(left, right, right_list)

# difflib package (method 2) - Haoran Wang

In [1]:
# Import

In [ ]:
import pandas as pd
import difflib

In [ ]:
# Data Preparation

In [ ]:
left_data = pd.read_csv("C:/Users/WHR/Desktop/final_project/group-project-spring-2024/Data/GroupProject/entity-resolution/left_dataset.csv")
right_data = pd.read_csv("C:/Users/WHR/Desktop/final_project/group-project-spring-2024/Data/GroupProject/entity-resolution/right_dataset.csv")

In [ ]:
left = pd.DataFrame(left_data)
right = pd.DataFrame(right_data)

In [ ]:
left['postal_code'] = left['postal_code'].astype(str)
right['zip_code'] = right['zip_code'].astype(str)

In [ ]:
left['postal_code'] = left['postal_code'].str[:5]
right['zip_code'] = right['zip_code'].str[:5]

In [ ]:
left['combined_address'] = left['address'] + ' ' + left['city'] + ' ' + left['state'] + ' ' + left['postal_code']
right['combined_address'] = right['address'] + ' ' + right['city'] + ' ' + right['state'] + ' ' + right['zip_code']

In [ ]:
for col in left.columns:
    left[col] = left[col].apply(lambda x: x.lower() if type(x) == str else x)
for col in right.columns:
    right[col] = right[col].apply(lambda x: x.lower() if type(x) == str else x)

In [ ]:
cities_left_count = left['city'].nunique()
cities_right_count = right['city'].nunique()
print(cities_left_count, cities_right_count)

In [ ]:
cities_left = left['city'].unique()
left_list = cities_left.tolist()

cities_right = right['city'].unique()
right_list = cities_right.tolist()

In [ ]:
# Processing

In [ ]:
def find_matches(df1, df2, key1_primary, key2_primary, key1_secondary, key2_secondary, threshold=0.8):
    matches = []
    for i, row1 in df1.iterrows():
        best_score = threshold
        best_match = None
        for j, row2 in df2.iterrows():
            primary1 = str(row1[key1_primary]) if pd.notna(row1[key1_primary]) else ''
            primary2 = str(row2[key2_primary]) if pd.notna(row2[key2_primary]) else ''
            score = difflib.SequenceMatcher(None, primary1, primary2).ratio()
            if score > best_score:
                best_score = score
                best_match = (row1['entity_id'], row2['business_id'], score)
        if best_match:
            matches.append(best_match)
        else:
            secondary1 = str(row1[key1_secondary]) if pd.notna(row1[key1_secondary]) else ''
            secondary2 = str(row2[key2_secondary]) if pd.notna(row2[key2_secondary]) else ''
            score = difflib.SequenceMatcher(None, secondary1, secondary2).ratio()
            if score > best_score:
                best_score = score
                best_match = (row1['entity_id'], row2['business_id'], score)
            if best_match:
                matches.append(best_match)
    return matches

In [ ]:
def match_all(df1, df2, city_list):
    
    all_matches = []
    
    for city in city_list:
        
        left_data = df1[df1['city'] == city]
        right_data = df2[df2['city'] == city]
        
        refined_matches = find_matches(left_data, right_data, 'name', 'name', 'combined_address', 'combined_address')
        all_matches.append(refined_matches)
        
    return all_matches

In [ ]:
result = match_all(left, right, right_list)

In [ ]:
count = 0
for match in result:
    count += len(match)
print(count)

In [ ]:
result

# fuzzywuzzy package - Yuanyuan Yue

In [ ]:
pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd

left_df = pd.read_csv('/Users/yvonneyue/Downloads/left_dataset.csv')
right_df = pd.read_csv('/Users/yvonneyue/Downloads/right_dataset.csv')

In [ ]:
left_df['full_address'] = left_df.apply(lambda row: f"{row['address']}, {row['postal_code']}", axis=1)
left_df.head()

In [ ]:
right_df['simple_zip_code'] = right_df['zip_code'].str.slice(0, 5)
right_df['simple_zip_code'] = right_df['simple_zip_code'].astype(int)
right_df['full_address'] = right_df.apply(lambda row: f"{row['address']}, {row['zip_code']}", axis=1)
right_df.head()

In [ ]:
def calculate_similarity(left, right):
    left = str(left['name']) + str(left['address'])
    right = str(right['name']) + str(right['address'])
    confidence_score = fuzz.token_sort_ratio(left, right)
    return confidence_score

In [2]:
# postal_code and zip_code in -19000

In [ ]:
left_df_19000 = left_df[left_df['postal_code'] <= 19000]
right_df_19000 = right_df[right_df['simple_zip_code'] <= 19000]

In [ ]:
len(left_df_19000)

In [ ]:
matches_19000 = []
for index_left, left in left_df_19000.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_19000.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_19000.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [ ]:
for match in matches_19000:
    match['confidence_score'] /= 100.0

In [ ]:
#matches_df_FL = pd.DataFrame(matches_FL)
flt_matches_19000 = [match for match in matches_19000 if match['confidence_score'] > 0.8]
flt_matches_19000[:5]

In [ ]:
len(flt_matches_19000)

In [ ]:
# postal_code and zip_code in 19001-19100

In [ ]:
left_df_19001_19100 = left_df[(left_df['postal_code'] >= 19001) & (left_df['postal_code'] <= 19100)]
right_df_19001_19100 = right_df[(right_df['simple_zip_code'] >= 19001) & (right_df['simple_zip_code'] <= 19100)]

In [ ]:
len(left_df_19001_19100)

In [ ]:
matches_19001_19100 = []
for index_left, left in left_df_19001_19100.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_19001_19100.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_19001_19100.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [ ]:
for match in matches_19001_19100:
    match['confidence_score'] /= 100.0

In [ ]:
flt_matches_19001_19100 = [match for match in matches_19001_19100 if match['confidence_score'] > 0.8]
flt_matches_19001_19100[:5]

In [ ]:
len(flt_matches_19001_19100)

In [ ]:
# postal_code and zip_code in 19101-19140

In [ ]:
left_df_19101_19140 = left_df[(left_df['postal_code'] >= 19100) & (left_df['postal_code'] <= 19140)]
right_df_19101_19140 = right_df[(right_df['simple_zip_code'] >= 19100) & (right_df['simple_zip_code'] <= 19140)]

In [ ]:
len(right_df_19101_19140)

In [ ]:
matches_19101_19140 = []
for index_left, left in left_df_19101_19140.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_19101_19140.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_19101_19140.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [ ]:
for match in matches_19101_19140:
    match['confidence_score'] /= 100.0

In [ ]:
flt_matches_19101_19140 = [match for match in matches_19101_19140 if match['confidence_score'] > 0.8]
flt_matches_19101_19140[:5]

In [ ]:
len(flt_matches_19101_19140)

In [ ]:
# postal_code and zip_code in 19141-19410

In [ ]:
left_df_19141_19410 = left_df[(left_df['postal_code'] >= 19141) & (left_df['postal_code'] <= 19410)]
right_df_19141_19410 = right_df[(right_df['simple_zip_code'] >= 19141) & (right_df['simple_zip_code'] <= 19410)]

In [ ]:
len(right_df_19141_19410)

In [ ]:
matches_19141_19410 = []
for index_left, left in left_df_19141_19410.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_19141_19410.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_19141_19410.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [ ]:
for match in matches_19141_19410:
    match['confidence_score'] /= 100.0

In [ ]:
flt_matches_19141_19410 = [match for match in matches_19141_19410 if match['confidence_score'] > 0.8]
flt_matches_19141_19410[:5]

In [ ]:
len(flt_matches_19141_19410)

In [ ]:
# postal_code and zip_code in 19411-33600

In [ ]:
left_df_19411_33600 = left_df[(left_df['postal_code'] >= 19411) & (left_df['postal_code'] <= 33600)]
right_df_19411_33600 = right_df[(right_df['simple_zip_code'] >= 19411) & (right_df['simple_zip_code'] <= 33600)]

In [ ]:
len(right_df_19411_33600)

In [ ]:
matches_19411_33600 = []
for index_left, left in left_df_19411_33600.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_19411_33600.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_19411_33600.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [ ]:
for match in matches_19411_33600:
    match['confidence_score'] /= 100.0

In [ ]:
flt_matches_19411_33600 = [match for match in matches_19411_33600 if match['confidence_score'] > 0.8]
flt_matches_19411_33600[:5]

In [ ]:
len(flt_matches_19411_33600)

In [ ]:
# postal_code and zip_code in 33601-33640

In [ ]:
left_df_33601_33640 = left_df[(left_df['postal_code'] >= 33601) & (left_df['postal_code'] <= 33640)]
right_df_33601_33640 = right_df[(right_df['simple_zip_code'] >= 33601) & (right_df['simple_zip_code'] <= 33640)]

In [ ]:
len(right_df_33601_33640)

In [ ]:
matches_33601_33640 = []
for index_left, left in left_df_33601_33640.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_33601_33640.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_33601_33640.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [ ]:
for match in matches_33601_33640:
    match['confidence_score'] /= 100.0

In [ ]:
flt_matches_33601_33640 = [match for match in matches_33601_33640 if match['confidence_score'] > 0.8]
flt_matches_33601_33640[:5]

In [ ]:
len(flt_matches_33601_33640)

In [ ]:
# postal_code and zip_code in 33641-34000

In [ ]:
left_df_33641_34000 = left_df[(left_df['postal_code'] >= 33641) & (left_df['postal_code'] <= 34000)]
right_df_33641_34000 = right_df[(right_df['simple_zip_code'] >= 33641) & (right_df['simple_zip_code'] <= 34000)]

In [ ]:
len(right_df_33641_34000)

In [ ]:
matches_33641_34000 = []
for index_left, left in left_df_33641_34000.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_33641_34000.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_33641_34000.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [ ]:
for match in matches_33641_34000:
    match['confidence_score'] /= 100.0

In [ ]:
flt_matches_33641_34000 = [match for match in matches_33641_34000 if match['confidence_score'] > 0.8]
flt_matches_33641_34000[:5]

In [ ]:
len(flt_matches_33641_34000)

In [ ]:
# postal_code and zip_code in 34001-37190

In [ ]:
left_df_34001_37190 = left_df[(left_df['postal_code'] >= 34001) & (left_df['postal_code'] <= 37190)]
right_df_34001_37190 = right_df[(right_df['simple_zip_code'] >= 34001) & (right_df['simple_zip_code'] <= 37190)]

In [ ]:
len(right_df_34001_37190)

In [ ]:
matches_34001_37190 = []
for index_left, left in left_df_34001_37190.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_34001_37190.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_34001_37190.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [ ]:
for match in matches_34001_37190:
    match['confidence_score'] /= 100.0

In [ ]:
flt_matches_34001_37190 = [match for match in matches_34001_37190 if match['confidence_score'] > 0.8]
flt_matches_34001_37190[:5]

In [ ]:
len(flt_matches_34001_37190)

In [ ]:
# postal_code and zip_code in 37191-46190

In [ ]:
left_df_37191_46190 = left_df[(left_df['postal_code'] >= 37191) & (left_df['postal_code'] <= 46190)]
right_df_37191_46190 = right_df[(right_df['simple_zip_code'] >= 37191) & (right_df['simple_zip_code'] <= 46190)]

In [ ]:
len(right_df_37191_46190)

In [ ]:
matches_37191_46190 = []
for index_left, left in left_df_37191_46190.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_37191_46190.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_37191_46190.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [ ]:
for match in matches_37191_46190:
    match['confidence_score'] /= 100.0

In [ ]:
flt_matches_37191_46190 = [match for match in matches_37191_46190 if match['confidence_score'] > 0.8]
flt_matches_37191_46190[:5]

In [ ]:
len(flt_matches_37191_46190)

In [ ]:
# postal_code and zip_code in 46191-63090

In [ ]:
left_df_46191_63090 = left_df[(left_df['postal_code'] >= 46191) & (left_df['postal_code'] <= 63090)]
right_df_46191_63090 = right_df[(right_df['simple_zip_code'] >= 46191) & (right_df['simple_zip_code'] <= 63090)]

In [ ]:
len(right_df_46191_63090)

In [ ]:
matches_46191_63090 = []
for index_left, left in left_df_46191_63090.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_46191_63090.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_46191_63090.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [ ]:
for match in matches_46191_63090:
    match['confidence_score'] /= 100.0

In [ ]:
flt_matches_46191_63090 = [match for match in matches_46191_63090 if match['confidence_score'] > 0.8]
flt_matches_46191_63090[:5]

In [ ]:
len(flt_matches_46191_63090)

In [ ]:
# postal_code and zip_code in 63091-

In [ ]:
left_df_63091 = left_df[left_df['postal_code'] >= 63091]
right_df_63091 = right_df[right_df['simple_zip_code'] >= 63091]

In [ ]:
len(right_df_63091)

In [ ]:
matches_63091 = []
for index_left, left in left_df_63091.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_63091.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_63091.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [ ]:
for match in matches_63091:
    match['confidence_score'] /= 100.0

In [ ]:
flt_matches_63091 = [match for match in matches_63091 if match['confidence_score'] > 0.8]
flt_matches_63091[:5]

In [ ]:
len(flt_matches_63091)

In [ ]:
# Sum all matching rows

In [ ]:
len(flt_matches_19000) + len(flt_matches_19001_19100) + len(flt_matches_19101_19140) + len(flt_matches_19141_19410) + len(flt_matches_19411_33600) + len(flt_matches_33601_33640) + len(flt_matches_33641_34000) + len(flt_matches_34001_37190) + len(flt_matches_37191_46190) + len(flt_matches_46191_63090) + len(flt_matches_63091)

In [ ]:
result = pd.concat([pd.DataFrame(flt_matches_19000), 
                    pd.DataFrame(flt_matches_19001_19100), 
                    pd.DataFrame(flt_matches_19101_19140),
                    pd.DataFrame(flt_matches_19141_19410),
                    pd.DataFrame(flt_matches_19411_33600),
                    pd.DataFrame(flt_matches_33601_33640),
                    pd.DataFrame(flt_matches_33641_34000),
                    pd.DataFrame(flt_matches_34001_37190),
                    pd.DataFrame(flt_matches_37191_46190),
                    pd.DataFrame(flt_matches_46191_63090),
                    pd.DataFrame(flt_matches_63091)
                   ], ignore_index=True)

In [ ]:
result.to_csv('output.csv', index=False)

# jellyfish package - Jake Papalia

In [ ]:
import pandas as pd
from multiprocessing import Pool
import jellyfish
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
print(dir(jellyfish))

In [ ]:
left_df = pd.read_csv("/Users/jakepapalia/Downloads/group-project-spring-2024 (1)/Data/GroupProject/entity-resolution/left_dataset.csv")
right_df = pd.read_csv("/Users/jakepapalia/Downloads/group-project-spring-2024 (1)/Data/GroupProject/entity-resolution/right_dataset.csv")

In [ ]:
# Preprocessing
left_df = left_df.fillna('')
right_df = right_df.fillna('')

left_df['name'] = left_df['name'].str.lower().str.replace('[^\w\s]', '').str.strip()
left_df['address'] = left_df['address'].str.lower().str.replace('[^\w\s]', '').str.strip()
left_df['city'] = left_df['city'].str.lower().str.replace('[^\w\s]', '').str.strip()
left_df['state'] = left_df['state'].str.lower().str.replace('[^\w\s]', '').str.strip()

right_df['name'] = right_df['name'].str.lower().str.replace('[^\w\s]', '').str.strip()
right_df['address'] = right_df['address'].str.lower().str.replace('[^\w\s]', '').str.strip()
right_df['city'] = right_df['city'].str.lower().str.replace('[^\w\s]', '').str.strip()
right_df['state'] = right_df['state'].str.lower().str.replace('[^\w\s]', '').str.strip()

subset_left = left_df.head(1000)
subset_right = right_df.head(1000)

In [ ]:
# function for matching chunks
def find_matches(left_chunk, right_chunk):
    matching_businesses = []
    for _, left_row in left_chunk.iterrows():
        for _, right_row in right_chunk.iterrows():
            # Calculate vectors
            vectorizer = TfidfVectorizer()
            left_vector = vectorizer.fit_transform([left_row['name'], left_row['address'], left_row['city'], left_row['state']])
            right_vector = vectorizer.transform([right_row['name'], right_row['address'], right_row['city'], right_row['state']])
            # Calculate cosine similarity
            similarity_score = cosine_similarity(left_vector, right_vector)[0][0]
            if similarity_score > 0.8:
                matching_businesses.append((left_row['entity_id'], right_row['business_id'], similarity_score))
    return matching_businesses

In [ ]:
chunk_size = 1000

# Process in chunks
matching_businesses = []
for left_start in range(0, len(left_df), chunk_size):
    left_chunk = left_df.iloc[left_start:left_start + chunk_size]
    for right_start in range(0, len(right_df), chunk_size):
        right_chunk = right_df.iloc[right_start:right_start + chunk_size]
        # Find matches for the current chunks
        matches = find_matches(left_chunk, right_chunk)
        matching_businesses.extend(matches)

matches_df = pd.DataFrame(matching_businesses, columns=['entity_id_left', 'business_id_right', 'similarity_score'])

# number of matches
print(len(matches_df))

# recordlinkage package - Enzo Zhang

In [ ]:
import pandas as pd

# Load the datasets
left_dataset_path = '/Users/haoranw/Desktop/5210 final project_Group_8/Data/left_dataset.csv'
right_dataset_path = '/Users/haoranw/Desktop/5210 final project_Group_8/Data/right_dataset.csv'

left_df = pd.read_csv(left_dataset_path)
right_df = pd.read_csv(right_dataset_path)

In [ ]:
# delete the columns of "size", "categroy"
left_df.drop(columns=['categories'], inplace=True)
right_df.drop(columns=['size'], inplace=True)

In [ ]:
# Change postal_code to zip_code
left_df['postal_code'] = left_df['postal_code'].apply(lambda x: str(int(x)) if pd.notna(x) else '')

left_df.rename(columns={'postal_code': 'zip_code'}, inplace=True)

# For the right dataset, split the 'zip_code' on the hyphen and take the first part
right_df['zip_code'] = right_df['zip_code'].apply(lambda x: x.split('-')[0].strip())

In [ ]:
# Trim the zip_code to the first five digits in both datasets, handling None values
left_df['zip_code'] = left_df['zip_code'].apply(lambda x: x[:5] if x and len(x) >= 5 else x)
right_df['zip_code'] = right_df['zip_code'].apply(lambda x: x[:5] if x and len(x) >= 5 else x)

left_df['zip_code'].head(), right_df['zip_code'].head()


In [ ]:
for column in left_df.columns:
    if pd.api.types.is_numeric_dtype(left_df[column]):
        left_df[column] = left_df[column].astype(str)

for column in right_df.columns:
    if pd.api.types.is_numeric_dtype(right_df[column]):
        right_df[column] = right_df[column].astype(str)



In [ ]:
#create block_key
left_df['block_key'] = (left_df['zip_code'].str[:3] + 
                        left_df['state'] + 
                        left_df['name'].str[0].str.upper())

right_df['block_key'] = (right_df['zip_code'].str[:3] + 
                         right_df['state'] + 
                         right_df['name'].str[0].str.upper())

# Ensure the new columns are strings and handle any potential missing values
left_df['block_key'] = left_df['block_key'].astype(str)
right_df['block_key'] = right_df['block_key'].astype(str)

In [ ]:
left_df

In [ ]:
right_df

In [ ]:
import pandas as pd
from unidecode import unidecode
import re

def normalize_text(column):
    column = column.astype(str)
    column = column.str.lower()
    column = column.apply(unidecode)
    column = column.str.replace(r'[^\w\s]', '', regex=True)
    return column


# Normalize business names
left_df['name'] = normalize_text(left_df['name'])
right_df['name'] = normalize_text(right_df['name'])

# Normalize addresses
left_df['address'] = normalize_text(left_df['address'])
right_df['address'] = normalize_text(right_df['address'])

In [ ]:
# remove the white space, and Standardize the address 

import pandas as pd
import re

def standardize_address(address):
    # Convert to string in case there are any non-string types
    address = str(address)
    # Remove any leading/trailing whitespace
    address = address.strip()
    # Replace common street suffix abbreviations with a standard
    street_suffixes = {
        'st.': 'Street', 'rd.': 'Road', 'ave.': 'Avenue',
        'dr.': 'Drive', 'ln.': 'Lane', 'blvd.': 'Boulevard'
    }
    for suffix in street_suffixes:
        address = re.sub(r'\b' + suffix + r'\b', street_suffixes[suffix], address, flags=re.IGNORECASE)
    # Standardize directional prefixes/suffixes
    directions = {
        'n ': 'North ', 's ': 'South ', 'e ': 'East ', 'w ': 'West ',
        'ne ': 'Northeast ', 'nw ': 'Northwest ', 'se ': 'Southeast ', 'sw ': 'Southwest '
    }
    for direction in directions:
        address = re.sub(r'\b' + direction, directions[direction], address, flags=re.IGNORECASE)
    # Standardize suite numbers
    address = re.sub(r'\bste\.? ', 'Suite ', address, flags=re.IGNORECASE)
    address = re.sub(r'\bapt\.? ', 'Apartment ', address, flags=re.IGNORECASE)
    # Remove all other special characters except for the hyphen and forward slash
    address = re.sub(r'[^\w\s/-]', '', address)
    # Replace multiple spaces with a single space
    address = re.sub(r'\s+', ' ', address)
    return address

# Apply the function to the address column of your DataFrames
left_df['address'] = left_df['address'].apply(standardize_address)
right_df['address'] = right_df['address'].apply(standardize_address)


In [ ]:
# Apply title case to the 'city' column of your DataFrames
left_df['city'] = left_df['city'].str.title()
right_df['city'] = right_df['city'].str.title()


In [ ]:
# add combine column

left_df['zip_code'] = left_df['zip_code'].astype(str)
right_df['zip_code'] = right_df['zip_code'].astype(str)

left_df['combined'] = left_df[['name', 'address', 'zip_code']].agg(' '.join, axis=1)
right_df['combined'] = right_df[['name', 'address', 'zip_code']].agg(' '.join, axis=1)


In [ ]:
left_df

In [ ]:
right_df

In [ ]:
import pandas as pd
import recordlinkage
from recordlinkage.index import Block

# Assuming previous steps are the same as provided (data loading, cleaning, and normalization)

# Using recordlinkage package

# Create an indexer object and use blocking on 'block_key'
indexer = Block('block_key')
pairs = indexer.index(left_df, right_df)

# Initialize the comparison object
compare = recordlinkage.Compare()

# Add comparison criteria. Here compare the 'combined' column using the Jaro-Winkler method.
compare.string('combined', 'combined', method='jarowinkler', threshold=0.8, label='combined_score')

# Compute the comparison vector
features = compare.compute(pairs, left_df, right_df)

# Filter matches based on the score threshold
matches = features[features['combined_score'] > 0.8]

matched_indices = matches.index

# Create a DataFrame from the matches with only necessary columns
results = [(left_id, right_id, score) for (left_id, right_id), score in matches.iterrows()]

# Display or save the results
print(results)


In [ ]:
import pandas as pd
import recordlinkage
from recordlinkage.index import Block

# Assuming previous steps are the same as provided (data loading, cleaning, and normalization)

# Using recordlinkage package

# Create an indexer object and use blocking on 'block_key'
indexer = Block('block_key')
pairs = indexer.index(left_df, right_df)

# Initialize the comparison object
compare = recordlinkage.Compare()

# Add comparison criteria. Here compare the 'combined' column using the Jaro-Winkler method.
compare.string('combined', 'combined', method='jarowinkler', threshold=0.85, label='combined_score')

# Compute the comparison vector
features = compare.compute(pairs, left_df, right_df)

# Filter matches based on the score threshold
matches = features[features['combined_score'] > 0.85]

# Create a DataFrame from the matches with only necessary columns
results = pd.DataFrame({
    'left_id': matches.index.get_level_values(0),
    'right_id': matches.index.get_level_values(1),
    'score': matches['combined_score']
})

# Save the results to a CSV file
results.to_csv('matching_results.csv', index=False)

print("Results have been saved to 'matching_results.csv'")


# tfidfvectorizer package - Jianfei Shi

In [ ]:
# Data Loaded & Preprossessed

In [ ]:
#Import packages
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Load the datasets
left_df = pd.read_csv('left_dataset.csv')
right_df = pd.read_csv('right_dataset.csv')

#Preprocessing function for the names and addresses
def preprocess(text):
    #Simple case normalization and stripping leading/trailing whitespace
    return text.lower().strip()

#Apply the preprocessing function to the name and address columns
left_df['name'] = left_df['name'].astype(str).apply(preprocess)
left_df['address'] = left_df['address'].astype(str).apply(preprocess)
right_df['name'] = right_df['name'].astype(str).apply(preprocess)
right_df['address'] = right_df['address'].astype(str).apply(preprocess)

In [ ]:
# Matching

In [ ]:
#Combine name and address for a complete profile
left_df['profile'] = left_df['name'] + ' ' + left_df['address']
right_df['profile'] = right_df['name'] + ' ' + right_df['address']

#Vectorize the profiles using TF-IDF
vectorizer = TfidfVectorizer()
left_tfidf = vectorizer.fit_transform(left_df['profile'])
right_tfidf = vectorizer.transform(right_df['profile'])

#Calculate cosine similarity between all profiles
similarity_matrix = cosine_similarity(left_tfidf, right_tfidf)

#Consider a pair a match if the cosine similarity is above a threshold (e.g., 0.8)
matches = []
threshold = 0.8
for i, row in enumerate(similarity_matrix):
    for j, score in enumerate(row):
        if score > threshold:
            matches.append((left_df.iloc[i]['entity_id'], right_df.iloc[j]['business_id'], score))

#Convert matches to DataFrame
matches_df = pd.DataFrame(matches, columns=['left_entity_id', 'right_business_id', 'confidence_score'])

#Sort by confidence score in descending order
matches_df = matches_df.sort_values(by='confidence_score', ascending=False)

In [ ]:
# Output Matches w/ High Degree of Confidence

In [ ]:
print(matches_df)

In [ ]:
#Save the matches to a CSV file
matches_df.to_csv('/Users/sterben/Library/Mobile Documents/com~apple~CloudDocs/Documents/apan5210/group-project-spring-2024/results1.csv', index=False)